In [ ]:
from keras.models import load_model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

In [ ]:
from numpy import save

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

In [ ]:
import os

In [ ]:
trial_model=load_model('C:\\Users\\Om\\Desktop\\Cerberus\\Utilities\\VGGFace.h5') #load the pre-trained VGGface model

In [ ]:
#Funtion that extracts the face features into numpy arrays
#featmodel: model used for prediction. VGGFace in this case. 
#crpimg: variable that stores the images and correct RGB values.
def features(featmodel, crpimg, transform=False): 
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:] #stores predictions
    return output

In [ ]:
#Function that accesses the required photo directories in sequence and extract face features.
#def feature_extractor(inpDir,person,timestep,sample): 
#inpDir: variable that stores path of photos directory
#person: variable that stores the person number **not a string value**
#sample: variable that stores the sample number **not a string value**
#timestep: variable that specifies the required timestep
def feature_extractor(inpDir,person,timestep,sample): 
    
    #if ex_type == True:
        #Y_train_labels=np.ones((5,2622)) 
    #else:
        #Y_train_labels=np.zeros((5,2622))  
        
      
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    #input dataset most likely to be People_cerebrus/photos
    linpDir = os.listdir(inpDir) #list all directories in dataset
    personStr= linpDir[person]
    sampleFolder = '%s\\%s' % (inpDir,personStr) #opening sample folder
    lsampleFolder = os.listdir(sampleFolder)
    i = 0
    utterFolder = '%s\\%s' % (sampleFolder,lsampleFolder[sample])#opening utterance folders
    lutterFolder = os.listdir(utterFolder)
    for utterances in lutterFolder:
        utterNumber= '%s\\%s' % (utterFolder,utterances)
        lutterNumber= os.listdir(utterNumber)
        frame = lutterNumber[timestep] #accessing images of required timestep
        i = i + 1
        image= "%s\\%s" % (utterNumber,frame) 
        im=Image.open(image)
        im = im.resize((224,224)) #resize required to pass through VGGFace
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dimensions for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
    feature_sequence_reshaped = feature_sequence[:,0,:,:] #removing unnecessary extra dimensions
    return feature_sequence_reshaped

    
    #WB_CNN= Sequential()
    #WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation= 'relu', input_shape=(1,1,2622)))
    #WB_CNN.add( Dropout(0.2) )
    #WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation='relu',input_shape=(1,1,2622)))
    #WB_CNN.add( Dropout(0.2) )
    #WB_CNN.add(Convolution2D(512, kernel_size=(1,1), activation='relu'))
    #WB_CNN.add(Flatten())   

    #WB_CNN.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    #WB_CNN.fit(feature_sequence,Y_train_labels,epochs=20,batch_size=2)
    #WB_CNN.save('Bottle_Neck.h5')
    #CNN_out = WB_CNN.predict(feature_sequence)
    #print(CNN_out.shape)
   

In [ ]:
#Function that extracts face features of all images across all timesteps for one word.
#word_extractor(inpDir,person,sample):
#inpDir: variable that stores path of photos directory
#person: variable that stores the person number **not a string value**
#sample: variable that stores the sample number **not a string value**
def word_extractor(inpDir,person,sample):
    timesteps=20 #optimum number of timesteps.Change can be made.
    condition_count=0 #
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,person,i,sample)#calling feature_extractor()
        condition_count = condition_count + 1
        if condition_count==1 :
            CNN_out_total= CNN_out # extra condtion is added to maintain same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
    return CNN_out_total     

In [ ]:
#Function that extracts face features of all images for the person.
#person_extractor(inpDir,person):
#inpDir: variable that stores path of photos directory
#person: variable that stores the person number **not a string value**
def person_extractor(InpDir,person):
    p_dataset=[] #array to store values
    condition_count=0
    no_of_words=3 #one positive and 2 negatives. Can be changed.
    for i in range(no_of_words):
        condition_count = condition_count +1
            CNN_out_total= word_extractor(InpDir,person)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dimensions for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
    return p_dataset 

In [ ]:
#Function to reshape the face features array into a LSTM type input

def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    
    count_word = 0
    for word in range(3):
        count_i=0
        for utterance in range(5):
            count_j=0
            for timestep in range(100*word,100*(word+1),5):
                timestep = timestep + utterance
                sample_mat_j=sample_mat[timestep]
                sample_mat_j=sample_mat_j.reshape((1,2622))
                if count_j==0:
                    feature_vector=sample_mat_j
                else:
                    feature_vector=np.concatenate((feature_vector,sample_mat_j))
                count_j = count_j + 1

            if count_i==0:
                feature_mat=feature_vector
            else:
                feature_mat= np.concatenate((feature_mat,feature_vector))
            count_i= count_i + 1
            print(feature_mat.shape)
            
        if count_word==0:
            LSTM_input=feature_mat
        else:
            LSTM_input= np.concatenate((LSTM_input,feature_mat))
        count_word= count_word + 1
                
    print(LSTM_input.shape)
  
    return LSTM_input

In [ ]:
def BottleNeck(InpDir):
    p_dataset=person_extractor(InpDir,0)
    p_dataset_LSTM=reshape_LSTM(p_dataset)
    p_total_dataset=p_dataset_LSTM
    print('person done')
    return p_total_dataset